In [46]:
import getpass
import os
from credentials import *
import tiktoken

In [4]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint= Azure_OpenAI_OB_Endpoint ,
    openai_api_version="2024-02-15-preview",
    model_name="gpt-4o",
    openai_api_key= Azure_OpenAI_OB_Key,
    openai_api_type="azure",
    temperature=0,
    deployment_name="gpt-4o-deploy",
    #streaming=True,
)

In [47]:
import tiktoken

def count_tokens(text, model="gpt-4o"):
    # Obtenir l'encodage en fonction du modèle
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        # Si le modèle n'est pas pris en charge, utilisez un encodage par défaut
        encoding = tiktoken.get_encoding("cl100k_base")
    
    # Convertir le texte en tokens
    tokens = encoding.encode(text)
    
    # Retourner le nombre de tokens
    return len(tokens)

# Exemple d'utilisation
my_text = "Voici un exemple de texte que nous voulons analyser."
token_count = count_tokens(my_text, model="gpt-4o")
print(f"Nombre de tokens : {token_count}")

Nombre de tokens : 10


## 1 - Test

In [5]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [("system",
            """Tu es un avocat et analyste juridique très expérimenté, tu as pour but d'analyser et commenter les changements 
            réglementaires de chaque article , tu trouveras ici l'ancienne et la nouvelle version de l'article que l on te donne. 
            Tu adopteras un language juridique précis. 
            Pour avoir du contexte , sache que ce texte est extrait du {code}"""),
        ("human", 
            "Voici l'ancienne version de l'article a analyser {old_version} et voici la nouvelle version du meme article {new_version}")]
)

In [6]:
from langchain_core.output_parsers import StrOutputParser
chain = prompt | llm | StrOutputParser()

contenu_ancien = "Les établissements de crédit, les sociétés de financement, les établissements de monnaie électronique qui fournissent des services de paiement, les établissements de paiement, les intermédiaires en opérations de banque et en services de paiement, les intermédiaires en financement participatif, les prestataires de services de financement participatif dans le cadre de leurs activités de facilitation d'octroi de prêts, les entreprises d'assurance dans le cadre de leur activité de prêts et les sociétés de gestion mentionnées au premier alinéa de l'article L. 519-2 qui recourent aux services d'intermédiaires en opérations de banque et en services de paiement doivent s'assurer que ceux-ci sont immatriculés conformément à l'article L. 519-3-1 . Les établissements de crédit, les sociétés de financement, les établissements de monnaie électronique qui fournissent des services de paiement, les établissements de paiement, les intermédiaires en financement participatif, les prestataires de services de financement participatif dans le cadre de leurs activités de facilitation d'octroi de prêts, les entreprises d'assurance dans le cadre de leur activité de prêts et les sociétés de gestion mentionnées au premier alinéa de l'article L. 519-2 qui recourent à des intermédiaires ressortissants d'un Etat membre de l'Union européenne ou d'un Etat partie à l'accord sur l'Espace économique européen autre que la France et exerçant sur le territoire français en régime de libre prestation de services ou de libre établissement s'assurent auprès de l'organisme qui tient le registre prévu au I de l'article L. 512-1 du code des assurances que ceux-ci ont effectué les formalités requises à l'article L. 519-9 du présent code."
contenu_nouveau = "Les établissements de crédit, les sociétés de financement, les établissements de monnaie électronique qui fournissent des services de paiement, les établissements de paiement, les intermédiaires en opérations de banque et en services de paiement, les intermédiaires en financement participatif, les entreprises d'assurance dans le cadre de leur activité de prêts et les sociétés de gestion mentionnées au premier alinéa de l'article L. 519-2 qui recourent aux services d'intermédiaires en opérations de banque et en services de paiement doivent s'assurer que ceux-ci sont immatriculés conformément à l'article L. 519-3-1 . Les établissements de crédit, les sociétés de financement, les établissements de monnaie électronique qui fournissent des services de paiement, les établissements de paiement, les intermédiaires en financement participatif, les entreprises d'assurance dans le cadre de leur activité de prêts et les sociétés de gestion mentionnées au premier alinéa de l'article L. 519-2 qui recourent à des intermédiaires ressortissants d'un Etat membre de l'Union européenne ou d'un Etat partie à l'accord sur l'Espace économique européen autre que la France et exerçant sur le territoire français en régime de libre prestation de services ou de libre établissement s'assurent auprès de l'organisme qui tient le registre prévu au I de l'article L. 512-1 du code des assurances que ceux-ci ont effectué les formalités requises à l'article L. 519-9 du présent code."
llm_output = chain.invoke(
    {
        "code": "Code monétaire et financier",
        "old_version": contenu_ancien,
        "new_version": contenu_nouveau,
        "input": "I love programming.",
    }
)

In [7]:
print(llm_output)

L'analyse des modifications apportées à l'article en question du Code monétaire et financier révèle une suppression notable. 

**Ancienne version :**
L'ancienne version de l'article incluait les "prestataires de services de financement participatif dans le cadre de leurs activités de facilitation d'octroi de prêts" parmi les entités qui doivent s'assurer que les intermédiaires en opérations de banque et en services de paiement sont immatriculés conformément à l'article L. 519-3-1. De plus, cette catégorie était également mentionnée dans le contexte des intermédiaires ressortissants d'un État membre de l'Union européenne ou d'un État partie à l'accord sur l'Espace économique européen exerçant en France en régime de libre prestation de services ou de libre établissement.

**Nouvelle version :**
Dans la nouvelle version de l'article, la mention des "prestataires de services de financement participatif dans le cadre de leurs activités de facilitation d'octroi de prêts" a été supprimée. Ain

## 2 - Function

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [9]:
def llm_legal_change_com_v1(old_version, new_version): 
    prompt = ChatPromptTemplate.from_messages([("system",
            """
            Tu es un avocat et analyste juridique très expérimenté, tu as pour but d'analyser et de commenter les changements 
            réglementaires de chaque article , tu trouveras ici l'ancienne et la nouvelle version de l'article que l on te donne. 
            Tu adopteras un language juridique précis. Tu n'inventeras rien et tu te baseras uniquement sur les données fournies.
            Si tu ne sais pas ou tu ne comprends pas , dis le. 
            Inutile de reciter les textes fournis, tu ne retourneras que ton analyse.
            """),
        ("human", 
            "Voici l'ancienne version de l'article a analyser {old_version} et voici la nouvelle version du meme article {new_version}")])
    
    chain = prompt | llm | StrOutputParser()
    
    llm_output = chain.invoke({
        "old_version": old_version,
        "new_version": new_version})
    
    return llm_output

In [10]:
# Amélioration prompt avec o1
def llm_legal_change_com_v2(old_version, new_version): 
    prompt = ChatPromptTemplate.from_messages([("system",                                  
        """
	Tu es un avocat et analyste juridique très expérimenté. 
    Ta mission consiste à examiner et commenter les évolutions réglementaires d’un article de loi en comparant sa version 
    antérieure avec sa version révisée.

	Consignes :
	1.	Utilise un langage juridique précis.
	2.	Base-toi uniquement sur les informations fournies, sans inventer ni extrapoler.
	3.	Si tu ne comprends pas un point ou si les informations sont insuffisantes, signale-le clairement.
	4.	Ne récite pas le texte en intégralité : concentre-toi sur l’analyse et la comparaison.

	Contenu à analyser :
	•	Ancienne version : {old_version}
	•	Nouvelle version : {new_version}

	Objectif :
    Produis un commentaire juridique concis, en mettant en évidence les principaux changements, leur portée et leurs éventuelles conséquences. N’inclus dans ta réponse que l’analyse finale.

            """)])
    
    chain = prompt | llm | StrOutputParser()
    
    llm_output = chain.invoke({
        "old_version": old_version,
        "new_version": new_version})
    
    return llm_output

## 3 - Data prep

In [12]:
import pandas as pd
AML5_modifs= pd.read_excel("/Users/oussa/Desktop/Github_perso/3 - Legal FR Tracker/data_output/Output_AML_5.xlsx")

In [13]:
# Filter on AML text
AML5_modifs =  AML5_modifs[AML5_modifs['Titre Article Modificateur'].str.contains("n°2020-1544")| AML5_modifs['Titre Article Modificateur'].str.contains("n°2020-115") ]

In [14]:
#check results
AML5_modifs['Titre Article Modificateur'].unique()

array(['Ordonnance n°2020-1544 du 9 décembre 2020 - art. 1',
       'Ordonnance n°2020-1544 du 9 décembre 2020 - art. 2',
       'Ordonnance n°2020-1544 du 9 décembre 2020 - art. 3',
       'Ordonnance n°2020-1544 du 9 décembre 2020 - art. 4',
       'Ordonnance n°2020-1544 du 9 décembre 2020 - art. 5',
       'Ordonnance n°2020-1544 du 9 décembre 2020 - art. 6',
       'Ordonnance n°2020-1544 du 9 décembre 2020 - art. 7',
       'Ordonnance n°2020-1544 du 9 décembre 2020 - art. 8',
       'Ordonnance n°2020-1544 du 9 décembre 2020 - art. 9',
       'Ordonnance n°2020-1544 du 9 décembre 2020 - art. 10',
       'Ordonnance n°2020-1544 du 9 décembre 2020 - art. 11',
       'Ordonnance n°2020-115 du 12 février 2020 - art. 1',
       'Ordonnance n°2020-115 du 12 février 2020 - art. 2',
       'Ordonnance n°2020-115 du 12 février 2020 - art. 3',
       'Ordonnance n°2020-115 du 12 février 2020 - art. 4',
       'Ordonnance n°2020-115 du 12 février 2020 - art. 5',
       'Ordonnance n°2020-1

In [15]:
# Keep only changed articles in their content
AML5_modifs = AML5_modifs[AML5_modifs['Compare contenu'] =="OK change"]
AML5_modifs['Compare contenu'].unique()

array(['OK change'], dtype=object)

In [16]:
# Delete KO in Old and new articles
AML5_modifs = AML5_modifs[~AML5_modifs['Contenu_Ancien_Article'].str.contains("KO", na=False)]
AML5_modifs = AML5_modifs[~AML5_modifs['Contenu_Nouv_Vers_Article'].str.contains("KO", na=False)]

In [17]:
len(AML5_modifs)

54

## 4 - Apply function for each row

In [18]:
#   AML5_modifs
#   llm_legal_change_com()

In [19]:
AML5_modifs_select = AML5_modifs.iloc[:10]

### 4.1 - Application fonction 1

In [20]:
AML5_modifs_select['LLM_Change_Analysis_1'] = AML5_modifs_select.apply(
    lambda row: llm_legal_change_com_v1(row['Contenu_Ancien_Article'], row['Contenu_Nouv_Vers_Article']), 
    axis=1)

/var/folders/ty/2g_hgfks4nl_84mz496fmn600000gn/T/ipykernel_68977/819962179.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AML5_modifs_select['LLM_Change_Analysis_1'] = AML5_modifs_select.apply(


In [21]:
AML5_modifs_select.to_excel("data_output_ipynb/output_llm_change_analysis_v1.xlsx")

### 4.2 - Application fonction 2

In [22]:
AML5_modifs_select['LLM_Change_Analysis_2'] = AML5_modifs_select.apply( 
        lambda row: llm_legal_change_com_v2(row['Contenu_Ancien_Article'], row['Contenu_Nouv_Vers_Article']), axis=1 ) 

/var/folders/ty/2g_hgfks4nl_84mz496fmn600000gn/T/ipykernel_68977/2587971769.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AML5_modifs_select['LLM_Change_Analysis_2'] = AML5_modifs_select.apply(


In [23]:
AML5_modifs_select.to_excel("data_output_ipynb/output_llm_change_analysis_v2.xlsx")

### 4.3 - Application fonction 3

In [24]:
AML5_modifs_select['LLM_Change_Analysis_3'] = AML5_modifs_select.apply( 
        lambda row: llm_legal_change_com_v3(row['Contenu_Ancien_Article'], row['Contenu_Nouv_Vers_Article']), axis=1 ) 

/var/folders/ty/2g_hgfks4nl_84mz496fmn600000gn/T/ipykernel_68977/2334415541.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AML5_modifs_select['LLM_Change_Analysis_3'] = AML5_modifs_select.apply(


In [25]:
AML5_modifs_select.to_excel("data_output_ipynb/output_llm_change_analysis_v3.xlsx")

## 5 - Summarize

In [30]:
all_llm_analysis = AML5_modifs_select['LLM_Change_Analysis_3'] 

In [31]:
all_llm_analysis = all_llm_analysis.to_string()

In [43]:
text_variable = AML5_modifs_select['LLM_Change_Analysis_3'].str.cat()
text_variable

In [48]:
token_count = count_tokens(text_variable, model="gpt-4o")
print(f"Nombre de tokens : {token_count}")

Nombre de tokens : 6025


### 5.1 - Simple summarize

In [57]:
# Amélioration prompt avec o1 + intération
def wrap_up_simple(text_to_summarize): 
    prompt = ChatPromptTemplate.from_messages([("system",                                  
        """
		Tu es un avocat et analyste juridique très expérimenté.
		Ta mission consiste à produire un résumé global des analyses réalisées sur les évolutions réglementaires de plusieurs articles de 
  		loi en comparant leurs versions antérieures et révisées.

	Consignes :
	1.	Utilise un langage juridique précis, clair et synthétique.
	2.	Appuie-toi uniquement sur les analyses fournies pour chaque article. Ne fais aucune supposition ou extrapolation.
	3.	Regroupe et hiérarchise les informations :
	•	Identifie les thématiques ou tendances communes (par exemple : renforcement des obligations, simplification de procédures, etc.).
	•	Mets en évidence les changements les plus significatifs et leur impact global.
	4.	Si certaines analyses révèlent des ambiguïtés ou des manques d’informations, mentionne-les brièvement.

	Objectif :
 		Fournis un résumé synthétique mettant en lumière les évolutions majeures, les implications globales, et les tendances observées 
   		à travers les différents articles. Structure ta réponse de manière organisée (par exemple : par thématique ou par impact).
     
    Voici le text a résumer : {text_to_summarize}
            """)])
    
    chain = prompt | llm | StrOutputParser()
    
    llm_output = chain.invoke({
        "text_to_summarize": text_to_summarize})
    
    return llm_output

In [58]:
summary_simple = wrap_up_simple(text_variable)


In [59]:
print(summary_simple)

### Résumé des Évolutions Réglementaires

#### Thématiques et Tendances Communes

1. **Suppression et Simplification des Obligations**
   - **Suppression de certaines obligations spécifiques** : Les nouvelles versions des articles de loi ont supprimé plusieurs obligations détaillées imposées aux prestataires de services, notamment en matière de sécurité, de gestion des conflits d'intérêts, et de résilience des systèmes informatiques. Cette suppression pourrait alléger la charge administrative et opérationnelle des prestataires, mais elle pourrait également réduire le niveau de protection et de sécurité offert aux clients.
   - **Simplification des exigences de conformité** : Les exigences détaillées concernant la conclusion de conventions avec les clients, la politique de conservation, la restitution des actifs numériques, et l'usage des clés cryptographiques ont été supprimées. Cette simplification pourrait faciliter l'entrée sur le marché pour les nouveaux prestataires, mais elle pou

### 5.2 - Custum summarize

In [61]:
# Amélioration prompt avec o1 + intération
def wrap_up_multi(text_to_summarize, audience: str, detail_level:str): 
    # audience: “professionnel” ou “tout public”
    # detail_level: “succinct” ou “détaillé”
    prompt = ChatPromptTemplate.from_messages([("system",                                  
        """
		Prompt amélioré avec niveaux de détail et public cible :

		Tu es un avocat et analyste juridique très expérimenté.
		Ta mission consiste à produire un résumé global des analyses réalisées sur les évolutions réglementaires de plusieurs articles de 
  		loi en comparant leurs versions antérieures et révisées.

	Consignes générales :
	1.	Utilise un langage juridique précis et adapté au public cible défini par la variable {audience} :
	•	Si {audience} est “professionnel”, adopte un ton technique et détaillé.
	•	Si {audience} est “tout public”, simplifie les termes juridiques pour les rendre accessibles tout en conservant leur précision.
 
	2.	Ajuste la longueur et le niveau de détail du résumé en fonction de la variable {detail_level} :
	•	Si {detail_level} est “succinct”, concentre-toi uniquement sur les points clés et les impacts majeurs.
	•	Si {detail_level} est “détaillé”, développe davantage les analyses, en incluant des exemples ou des explications supplémentaires lorsque pertinent.
 
	3.	Regroupe et hiérarchise les informations :
	•	Identifie les thématiques ou tendances communes (par exemple : renforcement des obligations, simplification de procédures, etc.).
	•	Mets en évidence les changements les plus significatifs et leur impact global.
	4.	Si certaines analyses révèlent des ambiguïtés ou des manques d’informations, mentionne-les brièvement.

	Variables :
	•	Niveau de détail : {detail_level} (valeurs possibles : “succinct” ou “détaillé”)
	•	Public cible : {audience} (valeurs possibles : “professionnel” ou “tout public”)

	Objectif :
		Fournis un résumé synthétique ou détaillé, selon le niveau demandé, mettant en lumière les évolutions majeures, 
  		les implications globales, et les tendances observées à travers les différents articles. Structure ta réponse de manière organisée 
    	(par exemple : par thématique ou par impact).
    
    Voici le text  : {text_to_summarize}
            """)])
    
    chain = prompt | llm | StrOutputParser()
    
    llm_output = chain.invoke({
        "text_to_summarize": text_to_summarize})
    
    return llm_output

In [64]:
resuult_multi = wrap_up_multi(text_variable, 'professionnel', 'succinct')

KeyError: "Input to ChatPromptTemplate is missing variables {'audience', 'detail_level'}.  Expected: ['audience', 'detail_level', 'text_to_summarize'] Received: ['text_to_summarize']\nNote: if you intended {audience} to be part of the string and not a variable, please escape it with double curly braces like: '{{audience}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "